# Import Raw File to OpenMSI

## Basic notebook code

In [ ]:
import sys
import os 
import ipywidgets as widgets
from IPython.display import display
import getpass

# Import main BASTet convert tool 
# sys.path.insert(0,"/project/projectdirs/openmsi/omsi_processing_status/bastet")
sys.path.insert(0,'/global/u2/b/bpb/repos/BASTet/')
# sys.path.insert(1,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )

from omsi.tools.convertToOMSI import main as convert_omsi
import omsi as omsi

from omsi.dataformat.omsi_file import *
# from omsi.analysis.multivariate_stats.omsi_cx import *
# from omsi.analysis.msi_filtering.ticNormalization import *
from omsi.analysis.findpeaks.omsi_findpeaks_global import *
from omsi.analysis.multivariate_stats.omsi_nmf import *
from omsi.analysis.multivariate_stats.omsi_kmeans import *


# try:
#     from omsi_server.omsi_access import views_helper
# except:
#     from omsi.omsi_access import views_helper

# Jupyter sets up logging so that log message are not displayed in the notebook, so we need to 
# reload the logging module in order to be able to have log messages appear in the notebook
import logging
reload(logging)
from omsi.shared.log import log_helper
log_helper.setup_logging()
log_helper.set_log_level('DEBUG')

In [ ]:
omsi_base = omsi.dataformat.file_reader_base

In [ ]:
omsi_base.file_reader_base.available_formats()

In [ ]:


out_filename = '/global/homes/b/bpb/Downloads/Flax_Pod_12_day_old_CS.h5'

f = omsi_file(out_filename , 'a' )
e = f.get_experiment(0) #All data is organized as experiments and this just gets the first one
d= e.get_msidata(0) #This gets the omsi_file_msidata object for the first raw dataset 
d.shape

peakCube = omsi_findpeaks_global(name_key='Global peak finding on raw data')
# myIons = np.asarray([844.304, 868.288, 852.303, 806.331, 828.312, 804.314])
peakCube.execute( msidata=d, mzdata=d.mz, integration_width=0.1, peakheight=20, smoothwidth=3, slwindow=100)
e.create_analysis(peakCube)
f.close_file()

In [ ]:
f = omsi_file(out_filename , 'a' )
e = f.get_experiment(0) #All data is organized as experiments and this just gets the first one
peak_cube = e.get_analysis(0)
nmf_data = omsi_nmf(name_key='NMF')
nmf_data.execute(msidata = peak_cube['peak_cube'],numComponents=20)
e.create_analysis(nmf_data)
f.close_file()

In [ ]:
f = omsi_file(out_filename , 'a' )
e = f.get_experiment(0) #All data is organized as experiments and this just gets the first one
peak_cube = e.get_analysis(0)
kmeans_data = omsi_kmeans(name_key='kmeans')
kmeans_data.execute(msidata = peak_cube['peak_cube'],numClusters=12)
e.create_analysis(kmeans_data)
f.close_file()